# Aviation Accidents Analysis

You are part of a consulting firm that is tasked to do an analysis of commercial and passenger jet airline safety. The client (an airline/airplane insurer) is interested in knowing what types of aircraft (makes/models) exhibit low rates of total destruction and low likelihood of fatal or serious passenger injuries in the event of an accident. They are also interested in any general variables/conditions that might be at play. Your analysis will be based off of aviation accident data accumulated from the years 1948-2023. 

Our client is only interested in airplane makes/models that are professional builds and could potentially still be active. Assume a max lifetime of 40 years for a make/model retirement and make sure to filter your data accordingly (i.e. from 1983 onwards). They would also like separate recommendations for small aircraft vs. larger passenger models. **In addition, make sure that claims that you make are statistically robust and that you have enough samples when making comparisons between groups.**


In this summative assessment you will demonstrate your ability to:
- **Use Pandas to load, inspect, and clean the dataset appropriately.**
- **Transform relevant columns to create measures that address the problem at hand.**
- conduct EDA: visualization and statistical measures to systematically understand the structure of the data
- recommend a set of airplanes and makes conforming to the client's request and identify at least *two* factors contributing to airplane safety. You must provide supporting evidence (visuals, summary statistics, tables) for each claim you make.

### Make relevant library imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data Loading and Inspection

### Load in data from the relevant directory and inspect the dataframe.
- inspect NaNs, datatypes, and summary statistics

In [9]:
df = pd.read_csv('AviationData.csv', encoding='latin1', low_memory=False)
df.head()


,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


## Data Cleaning

### Filtering aircrafts and events

We want to filter the dataset to include aircraft that the client is interested in an analysis of:
- inspect relevant columns
- figure out any reasonable imputations
- filter the dataset

In [10]:
# Inspect relevant columns
df[['Aircraft.Category', 'Amateur.Built', 'Event.Date']].info()

# Preview unique values to assess quality
print("Aircraft.Category:", df['Aircraft.Category'].unique())
print("Amateur.Built:", df['Amateur.Built'].unique())
print("Event.Date sample:", df['Event.Date'].head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Aircraft.Category  32287 non-null  object
 1   Amateur.Built      88787 non-null  object
 2   Event.Date         88889 non-null  object
dtypes: object(3)
memory usage: 2.0+ MB
Aircraft.Category: [nan 'Airplane' 'Helicopter' 'Glider' 'Balloon' 'Gyrocraft' 'Ultralight'
 'Unknown' 'Blimp' 'Powered-Lift' 'Weight-Shift' 'Powered Parachute'
 'Rocket' 'WSFT' 'UNK' 'ULTR']
Amateur.Built: ['No' 'Yes' nan]
Event.Date sample: 0    1948-10-24
1    1962-07-19
2    1974-08-30
3    1977-06-19
4    1979-08-02
Name: Event.Date, dtype: object


### Cleaning and constructing Key Measurables

Injuries and robustness to destruction are a key interest point for the client. Clean and impute relevant columns and then create derived fields that best quantifies what the client wishes to track. **Use commenting or markdown to explain any cleaning assumptions as well as any derived columns you create.**

**Construct metric for fatal/serious injuries**

*Hint:* Estimate the total number of passengers on each flight. The likelihood of serious / fatal injury can be estimated as a fraction from this.

In [11]:
# Convert Event.Date to datetime
df['Event.Date'] = pd.to_datetime(df['Event.Date'], errors='coerce')

# Filter step-by-step
filtered_df = df[
    (df['Aircraft.Category'] == 'Airplane') &
    (df['Amateur.Built'] == 'No') &
    (df['Event.Date'] >= '1983-01-01')
]

# Confirm result
filtered_df.shape


(21447, 31)

**Aircraft.Damage**
- identify and execute any cleaning tasks
- construct a derived column tracking whether an aircraft was destroyed or not.

In [14]:
# Inspect unique values in Aircraft.damage
filtered_df['Aircraft.damage'].value_counts(dropna=False)


Aircraft.damage
Substantial    16990
Destroyed       2316
NaN             1227
Minor            817
Unknown           97
Name: count, dtype: int64

### Investigate the *Make* column
- Identify cleaning tasks here
- List cleaning tasks clearly in markdown
- Execute the cleaning tasks
- For your analysis, keep Makes with a reasonable number (you can put the threshold at 50 though lower could work as well)

In [20]:
filtered_df = filtered_df.copy()
filtered_df['Destroyed'] = filtered_df['Aircraft.damage'].apply(lambda x: 1 if x == 'Destroyed' else 0)
filtered_df[['Aircraft.damage', 'Destroyed']].head()


                                                               


,Aircraft.damage,Destroyed
4149,Minor,0
4150,Minor,0
4171,Destroyed,1
4285,NaN,0
5957,Minor,0


### Inspect Model column
- Get rid of any NaNs.
- Inspect the column and counts for each model/make. Are model labels unique to each make?
- If not, create a derived column that is a unique identifier for a given plane type.

In [22]:
# Drop rows with missing model
filtered_df = filtered_df[filtered_df['Model'].notna()]

# Inspect overlap between Make and Model
print("Unique models:", filtered_df['Model'].nunique())
print("Unique make-model combos:", filtered_df[['Make', 'Model']].drop_duplicates().shape[0])

# Create unique identifier
filtered_df['Make_Model'] = filtered_df['Make'] + '_' + filtered_df['Model']

# Preview
filtered_df[['Make', 'Model', 'Make_Model']].head()


Unique models: 3553
Unique make-model combos: 5215


,Make,Model,Make_Model
4149,Lockheed,L-1011,Lockheed_L-1011
4150,Boeing,747,Boeing_747
4171,Piper,PA-28-140,Piper_PA-28-140
4285,De Havilland,DHC-6,De Havilland_DHC-6
5957,Douglas,DC-10-10,Douglas_DC-10-10


### Cleaning other columns
- there are other columns containing data that might be related to the outcome of an accident. We list a few here:
- Engine.Type
- Weather.Condition
- Number.of.Engines
- Purpose.of.flight
- Broad.phase.of.flight

Inspect and identify potential cleaning tasks in each of the above columns. Execute those cleaning tasks. 

**Note**: You do not necessarily need to impute or drop NaNs here.

In [23]:
# Check value counts and basic structure
columns_to_check = [
    'Engine.Type', 
    'Weather.Condition', 
    'Number.of.Engines', 
    'Purpose.of.flight', 
    'Broad.phase.of.flight'
]

for col in columns_to_check:
    print(f"\n{col}")
    print(filtered_df[col].value_counts(dropna=False))



Engine.Type
Engine.Type
Reciprocating      15014
NaN                 3943
Turbo Prop          1251
Turbo Fan            926
Turbo Jet            133
Unknown              133
Geared Turbofan       12
Turbo Shaft           11
Electric               5
UNK                    1
Name: count, dtype: int64

Weather.Condition
Weather.Condition
VMC    17082
NaN     2970
IMC     1068
Unk      215
UNK       94
Name: count, dtype: int64

Number.of.Engines
Number.of.Engines
1.0    15734
2.0     3004
NaN     2547
4.0       93
3.0       40
0.0        9
8.0        1
6.0        1
Name: count, dtype: int64

Purpose.of.flight
Purpose.of.flight
Personal                     11713
NaN                           3671
Instructional                 2753
Aerial Application             933
Business                       498
Unknown                        374
Positioning                    349
Skydiving                      164
Aerial Observation             161
Other Work Use                 155
Flight Test      

### Column Removal
- inspect the dataframe and drop any columns that have too many NaNs

In [25]:
import numpy as np

# Engine.Type
filtered_df['Engine.Type'] = filtered_df['Engine.Type'].str.strip().str.title()
filtered_df['Engine.Type'] = filtered_df['Engine.Type'].replace({'Unknown': np.nan, 'Unk': np.nan, 'UNK': np.nan})

# Weather.Condition
filtered_df['Weather.Condition'] = filtered_df['Weather.Condition'].str.strip().str.upper()
filtered_df['Weather.Condition'] = filtered_df['Weather.Condition'].replace({'UNK': np.nan, 'Unk': np.nan})

# Purpose.of.flight
filtered_df['Purpose.of.flight'] = filtered_df['Purpose.of.flight'].str.strip().str.title()
filtered_df['Purpose.of.flight'] = filtered_df['Purpose.of.flight'].replace({
    'Air Race show': 'Air Race/Show',
    'Air Race/show': 'Air Race/Show'
})
# Keep only columns with at least 20,000 non-null values
threshold = 20000
filtered_df = filtered_df.loc[:, filtered_df.notna().sum() >= threshold]

# Confirm shape and remaining columns
print("Remaining shape:", filtered_df.shape)
print("Remaining columns:", filtered_df.columns.tolist())


Remaining shape: (21429, 18)
Remaining columns: ['Event.Id', 'Investigation.Type', 'Accident.Number', 'Event.Date', 'Location', 'Country', 'Injury.Severity', 'Aircraft.damage', 'Aircraft.Category', 'Registration.Number', 'Make', 'Model', 'Amateur.Built', 'FAR.Description', 'Total.Uninjured', 'Publication.Date', 'Destroyed', 'Make_Model']


### Save DataFrame to csv
- its generally useful to save data to file/server after its in a sufficiently cleaned or intermediate state
- the data can then be loaded directly in another notebook for further analysis
- this helps keep your notebooks and workflow readable, clean and modularized

In [26]:
# Save the cleaned DataFrame to CSV
filtered_df.to_csv('cleaned_aviation_data.csv', index=False)


In [27]:
import os
'cleaned_aviation_data.csv' in os.listdir('.')


True

In [ ]:
true